In [1]:
import xarray as xr
import os
import netCDF4
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import random
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
X=np.load('/work/sds-lab/Shuochen/climsim/val_input.npy')
y=np.load('/work/sds-lab/Shuochen/climsim/val_target.npy')

X = torch.from_numpy(X).type(torch.float).to(device)
y = torch.from_numpy(y).type(torch.float).to(device)

In [4]:
LEARNING_RATE = 0.01
IN_FEATURES = 124
OUT_FEATURES = 128
RANDOM_SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=RANDOM_SEED)

# X_train = X_train.T
# X_test = X_test.T
# y_train = y_train.T
# y_test = y_test.T

In [5]:
# # Setup data loaders for batch
# train_dataset = TensorDataset(X_train, y_train)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# test_dataset = TensorDataset(X_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, hidden_dim = 64):
        super().__init__()

        self.linear1 = nn.Linear(124, hidden_dim)
        self.transformer = nn.Transformer(d_model = 64, nhead=4, num_encoder_layers=2)
        self.linear2 = nn.Linear(128, hidden_dim)

    def forward(self, x, y):
        x = self.linear1(x)
        print(x.shape)
        y = self.linear2(y)
        print(x.shape)
        out = self.transformer(x, y)
        return out

model = TransformerModel().to(device)

/home/wang.shuoc/.conda/envs/my_env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
torch.manual_seed(42)
epochs = 50

for epoch in range(epochs):

    model.train()
    y_pred = model(X_train, y_train)
    loss = loss_fn(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_pred = model(X_test, y_test)
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")


torch.Size([1153536, 64])
torch.Size([1153536, 64])


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.80 GiB. GPU 0 has a total capacity of 79.15 GiB of which 8.47 GiB is free. Process 47720 has 6.64 GiB memory in use. Process 48042 has 3.30 GiB memory in use. Including non-PyTorch memory, this process has 60.71 GiB memory in use. Of the allocated memory 60.20 GiB is allocated by PyTorch, and 17.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
y_train.shape

In [ ]:
X_train.shape

In [ ]:
a=nn.Linear(124,64)
a(X_train.to('cpu')).shape